# Prepare shapefiles

In [3]:
import arcpy
import os
arcpy.env.overwriteOutput = True
arcpy.env.addOutputsToMap=False
# Set the workspace for the input folder containing feature classes
input_workspace = r"T:\amandal2023$\Common_datasets\projected"
arcpy.env.workspace = input_workspace

# Set the output folder
output_folder = r"T:\amandal2023$\Working\SantaRosa\Datasets\impact_table\shp"

inundation = r"T:\amandal2023$\Working\SantaRosa\Datasets\impact_table\1d100yr_poly.shp"

# List all feature classes in the input workspace
feature_classes = arcpy.ListFeatureClasses()
for fc in feature_classes:
    if fc=='Water_body.shp':
        erase_fc=os.path.join(input_workspace, fc)

# Iterate through the list of feature classes
for fc in feature_classes:
    # Construct the full path of the feature class
    fc_path = os.path.join(input_workspace, fc)
    print(fc_path)
    #selection=arcpy.management.SelectLayerByLocation(fc_path, "intersect", inundation)
    if fc!='Water_body.shp':
        output_fc_tmp = os.path.join(output_folder, fc.replace('.shp','_temp.shp'))
        #arcpy.CopyFeatures_management(selection, output_fc_tmp)
        # Use the Clip tool
        arcpy.analysis.Clip(fc_path, inundation, output_fc_tmp)
        output_fc = os.path.join(output_folder, fc)
        # Perform the erase operation
        arcpy.analysis.Erase(output_fc_tmp, erase_fc, output_fc)
        arcpy.Delete_management(output_fc_tmp)
        del output_fc_tmp
    else:
        output_fc = os.path.join(output_folder, fc)
        arcpy.CopyFeatures_management(fc_path, output_fc)
    print(f"Selected and saved: {output_fc}")
    

print("Process completed.")


T:\amandal2023$\Common_datasets\projected\Federal_lands.shp
Selected and saved: T:\amandal2023$\Working\SantaRosa\Datasets\impact_table\shp\Federal_lands.shp
T:\amandal2023$\Common_datasets\projected\Indian_lands.shp
Selected and saved: T:\amandal2023$\Working\SantaRosa\Datasets\impact_table\shp\Indian_lands.shp
T:\amandal2023$\Common_datasets\projected\State_lands.shp
Selected and saved: T:\amandal2023$\Working\SantaRosa\Datasets\impact_table\shp\State_lands.shp
T:\amandal2023$\Common_datasets\projected\State_lands_NOS.shp
Selected and saved: T:\amandal2023$\Working\SantaRosa\Datasets\impact_table\shp\State_lands_NOS.shp
T:\amandal2023$\Common_datasets\projected\State_lands_OS.shp
Selected and saved: T:\amandal2023$\Working\SantaRosa\Datasets\impact_table\shp\State_lands_OS.shp
T:\amandal2023$\Common_datasets\projected\Water_body.shp
Selected and saved: T:\amandal2023$\Working\SantaRosa\Datasets\impact_table\shp\Water_body.shp
Process completed.


# Calculate Flooded Area

In [4]:
import arcpy
import os
import pandas as pd
arcpy.env.overwriteOutput = True
arcpy.env.addOutputsToMap=False

# Set the output folder
output_folder = r"T:\amandal2023$\Working\SantaRosa\Datasets\impact_table\temp"

Hucs = r"T:\amandal2023$\Working\SantaRosa\Datasets\Admin\Santa_Rosa_HUC12s.shp"

fc_path= r"T:\amandal2023$\Working\SantaRosa\Datasets\impact_table\1d100yr_poly.shp"


data_list = []
# Create a search cursor to iterate through the rows
with arcpy.da.SearchCursor(Hucs, ['SHAPE@','HUC12','AREAACRES','DESCRIPT']) as cursor:#'DESCRIPT'
    for row in cursor:
        name=row[1]
        huc_name=row[3]
        # Construct the full path of the feature class
        #fc_path = os.path.join(input_workspace, fc)
        clip_path=os.path.join(output_folder, 'temp_flood')
        #print(fc_path)
        # Perform the clip operation
        arcpy.analysis.Clip(fc_path, row[0], clip_path)

        # Add a new field for acres
        field_name = "acres"
        arcpy.AddField_management(clip_path, field_name, "DOUBLE")

        # Calculate the area in acres
        expression = "!SHAPE.area@ACRES!"
        arcpy.CalculateField_management(clip_path, field_name, expression, "PYTHON")

        # Create a SQL expression to select rows with an area greater than 10 acres
        sql_expression = f"{field_name} > 10"

        # Select rows based on the SQL expression
        arcpy.SelectLayerByAttribute_management(clip_path, "NEW_SELECTION", sql_expression)

        # Calculate the total area of selected rows
        selected_area = 0
        with arcpy.da.SearchCursor(clip_path, [field_name]) as cursor:
            for row1 in cursor:
                selected_area += row1[0]

        # Print the total area of selected rows
        print(f"Total area of flood in {name}: {selected_area} acres")
        data_list.append({"HUC": row[1],"Name":row[3],"Area(acres)": row[2], "Flooded_Area(acres)": selected_area})

        # Clear the selection
        arcpy.SelectLayerByAttribute_management(clip_path, "CLEAR_SELECTION")
        arcpy.Delete_management(clip_path)
        #data = pd.DataFrame({"Layer":a,"Area(acers)":selected_area }, index=range(len(feature_classes)))
           
df = pd.DataFrame(data_list)
print("Process completed.")

Total area of flood in 031401050500: 22.482524986925696 acres
Total area of flood in 031403050503: 9083.827497502545 acres
Total area of flood in 031401040501: 239.2663894922 acres
Total area of flood in 031401050201: 1059.491885565166 acres
Total area of flood in 031401040403: 434.758476961139 acres
Total area of flood in 031401040404: 636.0173341997479 acres
Total area of flood in 031401040304: 1672.471190053251 acres
Total area of flood in 031401050202: 689.4768743475324 acres
Total area of flood in 031401040406: 1443.293222062408 acres
Total area of flood in 031401050105: 889.1612452850612 acres
Total area of flood in 031401030804: 4521.9485591621315 acres
Total area of flood in 031401030805: 2055.994941572211 acres
Total area of flood in 031401050205: 3068.7480657664 acres
Total area of flood in 031403050403: 5889.5759426759105 acres
Total area of flood in 031401050303: 3020.013286764644 acres
Total area of flood in 031403050402: 1281.016284449951 acres
Total area of flood in 0314

In [5]:
df.to_excel(r'T:\amandal2023$\Working\SantaRosa\Datasets\impact_table\table1.xls')

[5]:1: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.


# Calculate Areas of different lands

In [4]:
import arcpy
import os
import pandas as pd
arcpy.env.overwriteOutput = True
arcpy.env.addOutputsToMap=False
# Set the workspace for the input folder containing feature classes
input_workspace = r"T:\amandal2023$\Working\SantaRosa\Datasets\impact_table\shp"
arcpy.env.workspace = input_workspace

# Set the output folder
output_folder = r"T:\amandal2023$\Working\SantaRosa\Datasets\impact_table\temp"

Hucs = r"T:\amandal2023$\Working\SantaRosa\Datasets\Admin\Santa_Rosa_HUC12s.shp"


        
    
    
# List all feature classes in the input workspace
feature_classes = arcpy.ListFeatureClasses()
i=0
df1 = pd.DataFrame()
# Create a search cursor to iterate through the rows
with arcpy.da.SearchCursor(Hucs, ['SHAPE@','HUC12','DESCRIPT']) as cursor:#'DESCRIPT'
    for row in cursor:
        df3 = pd.DataFrame()
        #data_list = []
        empty_dict = {}
        excel_name=str(row[1])+str(row[2])+'.csv'
        #print(row[1])
        suffix=str(row[1])
        #data_list.append({"HUC": row[1], "Name":row[2]})
        for fc in feature_classes:
            fc_path = os.path.join(input_workspace, fc)
            a,b=fc.split('.')
            #print(fc_path)
            clip_path=os.path.join(output_folder, fc.replace('.shp','-'+suffix+'.shp'))
            arcpy.analysis.Clip(fc_path, row[0], clip_path)
            print('clipped ',fc)
            field_name = "acres"
            arcpy.AddField_management(clip_path, field_name, "DOUBLE")
            # Calculate the area in acres
            expression = "!SHAPE.area@ACRES!"
            arcpy.CalculateField_management(clip_path, field_name, expression, "PYTHON")
            # Create a SQL expression to select rows with an area greater than 10 acres
            sql_expression = f"{field_name} > 10"
            # Select rows based on the SQL expression
            selection=arcpy.SelectLayerByAttribute_management(clip_path, "NEW_SELECTION", sql_expression)
            output_fc_tmp = os.path.join(output_folder, fc.replace('.shp','_temp.shp'))
            arcpy.CopyFeatures_management(selection, output_fc_tmp)
            # Calculate the total area of selected rows
            selected_area = 0
            with arcpy.da.SearchCursor(output_fc_tmp, [field_name]) as cursor:
                for row1 in cursor:
                    selected_area += row1[0]

            # Print the total area of selected rows
            print(f"Total area of {fc}: {selected_area} acres")
            arcpy.Delete_management(clip_path)
            arcpy.Delete_management(output_fc_tmp)
            #data_list.append({"HUC": row[1],"Layer": a, "Area(acres)": selected_area})
            #data_list.append({str(a)+"Area(acres)": selected_area})
            empty_dict[str(a)+"Area(acres)"]=selected_area
            
        

        #print(name)
        #print(excel_name)
        empty_dict["HUC"]=row[1]
        empty_dict["Name"]=row[2]
        #print(empty_dict)
        #data_list.append({"HUC": row[1], "Name":row[2]})
        df2 = pd.DataFrame([empty_dict])
        print(df2)
        #df3=df3.append({"HUC": row[1], "Name":row[2]},ignore_index=True)
        #df1=df1.append(df3)
        df1=df1.append(df2)
        #excel_path=os.path.join(output_folder,excel_name)
        #df.to_csv(excel_path)
#df2 = pd.DataFrame(data_list)
print("Process completed.")
           
            

        


clipped  Federal_lands.shp
Total area of Federal_lands.shp: 0 acres
clipped  Indian_lands.shp
Total area of Indian_lands.shp: 0 acres
clipped  State_lands.shp
Total area of State_lands.shp: 0 acres
clipped  State_lands_NOS.shp
Total area of State_lands_NOS.shp: 0 acres
clipped  State_lands_OS.shp
Total area of State_lands_OS.shp: 0 acres
clipped  Water_body.shp
Total area of Water_body.shp: 243586.66049713135 acres
   Federal_landsArea(acres)  ...           Name
0                         0  ...  PENSACOLA BAY

[1 rows x 8 columns]


[4]:77: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.


clipped  Federal_lands.shp
Total area of Federal_lands.shp: 0 acres
clipped  Indian_lands.shp
Total area of Indian_lands.shp: 0 acres
clipped  State_lands.shp
Total area of State_lands.shp: 0 acres
clipped  State_lands_NOS.shp
Total area of State_lands_NOS.shp: 0 acres
clipped  State_lands_OS.shp
Total area of State_lands_OS.shp: 0 acres
clipped  Water_body.shp
Total area of Water_body.shp: 10254.2495792605 acres
   Federal_landsArea(acres)  ...           Name
0                         0  ...  DELANEY RIVER

[1 rows x 8 columns]


[4]:77: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.


clipped  Federal_lands.shp
Total area of Federal_lands.shp: 0 acres
clipped  Indian_lands.shp
Total area of Indian_lands.shp: 0 acres
clipped  State_lands.shp
Total area of State_lands.shp: 0 acres
clipped  State_lands_NOS.shp
Total area of State_lands_NOS.shp: 0 acres
clipped  State_lands_OS.shp
Total area of State_lands_OS.shp: 0 acres
clipped  Water_body.shp
Total area of Water_body.shp: 1948.3154767223 acres
   Federal_landsArea(acres)  ...                               Name
0                         0  ...  UPPER POND CREEK-BLACKWATER RIVER

[1 rows x 8 columns]


...

clipped  Federal_lands.shp
Total area of Federal_lands.shp: 44.8969501964 acres
clipped  Indian_lands.shp
Total area of Indian_lands.shp: 0 acres
clipped  State_lands.shp
Total area of State_lands.shp: 0 acres
clipped  State_lands_NOS.shp
Total area of State_lands_NOS.shp: 0 acres
clipped  State_lands_OS.shp
Total area of State_lands_OS.shp: 0 acres
clipped  Water_body.shp
Total area of Water_body.shp: 626.4839699367001 acres
   Federal_landsArea(acres)  ...                      Name
0                  44.89695  ...  SANTA ROSA SOUND FRONTAL

[1 rows x 8 columns]


[4]:77: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.


clipped  Federal_lands.shp
Total area of Federal_lands.shp: 0 acres
clipped  Indian_lands.shp
Total area of Indian_lands.shp: 0 acres
clipped  State_lands.shp
Total area of State_lands.shp: 282.58651517 acres
clipped  State_lands_NOS.shp
Total area of State_lands_NOS.shp: 0 acres
clipped  State_lands_OS.shp
Total area of State_lands_OS.shp: 282.58651517 acres
clipped  Water_body.shp
Total area of Water_body.shp: 1600.2995449057998 acres
   Federal_landsArea(acres)  Indian_landsArea(acres)  ...           HUC         Name
0                         0                        0  ...  031401040204  PENNY CREEK

[1 rows x 8 columns]


[4]:77: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.


Process completed.


In [5]:
df1.to_excel(r'T:\amandal2023$\Working\SantaRosa\Datasets\impact_table\table2.xls')

[5]:1: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.


In [7]:
import pandas as pd
df1=pd.read_excel(r'T:\amandal2023$\Working\SantaRosa\Datasets\impact_table\table2.xls')
df=pd.read_excel(r'T:\amandal2023$\Working\SantaRosa\Datasets\impact_table\table1.xls')
merged_df = pd.merge(df, df1, left_on='HUC', right_on='HUC', how='inner')
merged_df.to_excel(r'T:\amandal2023$\Working\SantaRosa\Datasets\impact_table\table_all1.xls')

[7]:5: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
